In [1]:
import os
from azureml.core import Workspace
from azureml.core import Environment
from azureml.core import ScriptRunConfig, Experiment
from azureml.widgets import RunDetails
from azureml.core import Model
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core.runconfig import MpiConfiguration

In [2]:
# AMLワークスペースへの接続
ws = Workspace.from_config()

In [3]:
# training script
source_dir = "train/bert"
script_name = "dist_train_bert.py"

# environment file
environment_file = os.path.join("train", "bert", "dist_train_bert_env.yml")

# azure ml settings
environment_name = "pl-env-pt160-lang"
experiment_name = "dist-bert-livedoor"
compute_name = "gpu-cluster01"

In [4]:
# 学習環境作成、初回のみ長時間
env = Environment.from_conda_specification(environment_name, environment_file)

env.docker.enabled = True
env.docker.base_image = (
    "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.2-cudnn8-ubuntu18.04"
)

FileNotFoundError: [Errno 2] No such file or directory: 'train/resnet/dist_train_bert_env.yml'

In [ ]:
# 学習設定
cluster = ws.compute_targets[compute_name]

#dist_config = PyTorchConfiguration(node_count=2)
dist_config = MpiConfiguration(node_count=2)

src = ScriptRunConfig(
    source_directory=source_dir,
    script=script_name,
    arguments=[
        "--batch_size", 128,
        "--max_epochs", 10,
        "--gpus", 1,
        "--accelerator", "ddp",
        "--num_nodes", 2,],
    compute_target=cluster,
    environment=env,
    distributed_job_config=dist_config,
)

In [ ]:
# 実行
run = Experiment(ws, experiment_name).submit(src)
run.wait_for_completion(show_output=True)

In [ ]:
# モデル登録
run.register_model(
    model_name="bert-livedoor-model",
    model_path=os.path.join('outputs', 'model.onnx'),
    model_framework=Model.Framework.ONNX,
)